In [2]:
from copy import deepcopy

import bilby
import numpy as np
from tqdm.auto import trange

In [ ]:
"""
Tutorial to demonstrate running parameter estimation on a reduced parameter
space for an injected signal, using the relative binning likelihood.

This example estimates the masses using a uniform prior in both component masses
and distance using a uniform in comoving volume prior on luminosity distance
between luminosity distances of 100Mpc and 5Gpc, the cosmology is Planck15.
"""

# Set the duration and sampling frequency of the data segment that we're
# going to inject the signal into
duration = 120.0
sampling_frequency = 2048.0
minimum_frequency = 20

# Specify the output directory and the name of the simulation.
outdir = "outdir1"
label = "relative"
bilby.core.utils.setup_logger(outdir=outdir, label=label)

# Set up a random seed for result reproducibility.  This is optional!
np.random.seed(88170235)

# We are going to inject a binary black hole waveform.  We first establish a
# dictionary of parameters that includes all of the different waveform
# parameters, including masses of the two black holes (mass_1, mass_2),
# spins of both black holes (a, tilt, phi), etc.
injection_parameters = dict(
    mass_1=20,
    mass_2=3,
    a_1=0.4,
    a_2=0.3,
    tilt_1=0.5,
    tilt_2=1.0,
    phi_12=1.7,
    phi_jl=0.3,
    luminosity_distance=300.0,
    theta_jn=0.4,
    psi=2.7,
    phase=1.3,
    geocent_time=1126259642.413,
    ra=1.375,
    dec=-1.2108,
    fiducial=1,
)

# Fixed arguments passed into the source model
waveform_arguments = dict(
    waveform_approximant="IMRPhenomXPHM",
    reference_frequency=50.0,
    minimum_frequency=minimum_frequency,
)

# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    # frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole_relative_binning,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,
)

# Set up interferometers.  In this case we'll use two interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1). These default to their design
# sensitivity
ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] - 60,
)
ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
)

# Set up a PriorDict, which inherits from dict.
# By default we will sample all terms in the signal models.  However, this will
# take a long time for the calculation, so for this example we will set almost
# all of the priors to be equall to their injected values.  This implies the
# prior is a delta function at the true, injected value.  In reality, the
# sampler implementation is smart enough to not sample any parameter that has
# a delta-function prior.
# The above list does *not* include mass_1, mass_2, theta_jn and luminosity
# distance, which means those are the parameters that will be included in the
# sampler.  If we do nothing, then the default priors get used.
priors = bilby.gw.prior.BBHPriorDict()
for key in [
    "a_1","a_2","tilt_1","tilt_2","phi_12","phi_jl","psi",
    "ra","dec","geocent_time","phase"]:
    priors[key] = injection_parameters[key]

priors["mass_2"] = bilby.core.prior.base.Constraint(minimum=2, maximum=5, latex_label='$m_2$', unit=None)
priors["mass_ratio"] = bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.05, maximum=0.5, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False)
priors["chirp_mass"] = bilby.gw.prior.UniformInComponentsChirpMass(minimum = 1, maximum = 25, name="chirp+mass", latex_label = '$M_c$')
priors["luminosity_distance"] = bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None)
print(priors)
# Perform a check that the prior does not extend to a parameter space longer than the data
priors.validate_prior(duration, minimum_frequency)

# Set up the fiducial parameters for the relative binning likelihood to be the
# injected parameters. Note that because we sample in chirp mass and mass ratio
# but injected with mass_1 and mass_2, we need to convert the mass parameters
fiducial_parameters = injection_parameters.copy()
m1 = fiducial_parameters.pop("mass_1")
m2 = fiducial_parameters.pop("mass_2")
fiducial_parameters["chirp_mass"] = bilby.gw.conversion.component_masses_to_chirp_mass(
    m1, m2)
fiducial_parameters["mass_ratio"] = m2 / m1

# Initialise the likelihood by passing in the interferometer data (ifos) and
# the waveform generator
likelihood = bilby.gw.likelihood.RelativeBinningGravitationalWaveTransient(
    interferometers=ifos,
    waveform_generator=waveform_generator,
    priors=priors,
    distance_marginalization=True,
    fiducial_parameters=fiducial_parameters, chi=10)

result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler="dynesty",
    npoints=500,
    injection_parameters=injection_parameters,
    outdir=outdir,
    label=label,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
)

alt_waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    # frequency_domain_source_model=lal_binary_black_hole_relative_binning,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,
)
alt_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos,
    waveform_generator=alt_waveform_generator,
)
likelihood.distance_marginalization = False
weights = list()
for ii in trange(len(result.posterior)):
    parameters = dict(result.posterior.iloc[ii])
    likelihood.parameters.update(parameters)
    alt_likelihood.parameters.update(parameters)
    weights.append(
        alt_likelihood.log_likelihood_ratio() - likelihood.log_likelihood_ratio()
    )
weights = np.exp(weights)
print(
    f"Reweighting efficiency is {np.mean(weights)**2 / np.mean(weights**2) * 100:.2f}%"
)
print(f"Binned vs unbinned log Bayes factor {np.log(np.mean(weights)):.2f}")

# Generate result object with the posterior for the regular likelihood using
# rejection sampling
alt_result = deepcopy(result)
keep = weights > np.random.uniform(0, max(weights), len(weights))
alt_result.posterior = result.posterior.iloc[keep]

# Make a comparison corner plot.
bilby.core.result.plot_multiple(
    [result, alt_result],
    labels=["Binned", "Reweighted"],
    filename=f"{outdir}/{label}_corner.png",
)

12:19 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole_relative_binning
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
12:19 bilby INFO    : Injected signal in H1:
12:19 bilby INFO    :   optimal SNR = 22.11
12:19 bilby INFO    :   matched filter SNR = 20.90-0.24j
12:19 bilby INFO    :   mass_1 = 20
12:19 bilby INFO    :   mass_2 = 3
12:19 bilby INFO    :   a_1 = 0.4
12:19 bilby INFO    :   a_2 = 0.3
12:19 bilby INFO    :   tilt_1 = 0.5
12:19 bilby INFO    :   tilt_2 = 1.0
12:19 bilby INFO    :   phi_12 = 1.7
12:19 bilby INFO    :   phi_jl = 0.3
12:19 bilby INFO    :   luminosity_distance = 300.0
12:19 bilby INFO    :   theta_jn = 0.4
12:19 bilby INFO    :   psi = 2.7
12:19 bilby INFO    :   phase = 1.3
12:19 bilby INFO    :   geocent_time = 1126259642.413
12:19 bilby INFO    :   ra = 1.375
12:19 bilby INFO    :   dec = -1.2108
12:19 bilby IN

{'mass_1': Constraint(minimum=5, maximum=100, name='mass_1', latex_label='$m_1$', unit=None), 'mass_2': Constraint(minimum=2, maximum=5, name=None, latex_label='$m_2$', unit=None), 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.05, maximum=0.5, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False), 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=1, maximum=25, name='chirp+mass', latex_label='$M_c$', unit=None, boundary=None), 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology=FlatLambdaCDM(name="Planck15", H0=67.74 km / (Mpc s), Om0=0.3075, Tcmb0=2.7255 K, Neff=3.046, m_nu=[0.   0.   0.06] eV, Ob0=0.0486), name='luminosity_distance', latex_label='$d_L$', unit=Unit("Mpc"), boundary=None), 'dec': -1.2108, 'ra': 1.375, 'theta_jn': Sine(minimum=0, maximum=3.141592653589793, name='theta_jn', latex_label='$\\theta_{JN}$', unit=None, boundary=None), 'psi': 2.7, 'phase': 1.3, 'a_1'

12:19 bilby INFO    : Loaded distance marginalisation lookup table from .distance_marginalization_lookup.npz.
12:19 bilby INFO    : Initial fiducial waveforms set up
12:19 bilby INFO    : Summary Data Obtained
12:19 bilby INFO    : Fiducial likelihood: 383.99
12:19 bilby INFO    : Running for label 'relative', output will be saved to 'outdir1'
12:19 bilby WARNING : Parameter fiducial has no default prior and is set to 0, this will not be sampled and may cause an error.
12:19 bilby INFO    : Using lal version 7.1.6
12:19 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.3;Id: 2a94c13d8a2ba99cbdfda5425fbd32fb904cfb46;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
12:19 bilby INFO    : Using lalsimulation version 3.1.1
12:19 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.3;Id: 2a94c13d8a2ba99cbdfda5425fbd32fb904cfb46;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree

[notice] A new rele

3012it [00:00, ?it/s]

12:27 bilby INFO    : Written checkpoint file outdir1/relative_resume.pickle
12:38 bilby INFO    : Written checkpoint file outdir1/relative_resume.pickle
12:46 bilby INFO    : Written checkpoint file outdir1/relative_resume.pickle
12:46 bilby INFO    : Rejection sampling nested samples to obtain 1639 posterior samples
12:46 bilby INFO    : Sampling time: 0:53:44.820696


...

  0%|          | 0/1639 [00:00<?, ?it/s]

12:46 bilby INFO    : Computing SNRs for every sample.


  0%|          | 0/1639 [00:00<?, ?it/s]

12:47 bilby INFO    : Summary of results:
nsamples: 1639
ln_noise_evidence: -241107.381
ln_evidence: -240741.227 +/-  0.237
ln_bayes_factor: 366.153 +/-  0.237

12:47 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


  0%|          | 0/1639 [00:00<?, ?it/s]